In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objs as go
import copy

In [14]:
tz_wv1 = pd.read_stata("Data/lsms/Tanzania/TZA_2008_NPS1_v02_M_STATA_English_labels/HH.Geovariables_Y1.dta")
tz_wv2 = pd.read_stata("Data/lsms/Tanzania/TZA_2010_NPS-R2_v02_M_STATA8/HH.Geovariables_Y2.dta")
tz_wv2_a = pd.read_stata("Data/lsms/Tanzania/TZA_2010_NPS-R2_v02_M_STATA8/HH_SEC_A.dta")
tz_wv3 = pd.read_csv("Data/lsms/Tanzania/TZA_2012_NPS-R3_v01_M_CSV/HouseholdGeovars_Y3.csv")
tz_wv3_a = pd.read_csv("Data/lsms/Tanzania/TZA_2012_NPS-R3_v01_M_CSV/HH_SEC_A.csv")
tz_wv3_sec_a = pd.read_csv("Data/lsms/Tanzania/TZA_2012_NPS-R3_v01_M_CSV/HH_SEC_A.csv")
tz_wv4 = pd.read_csv("Data/lsms/Tanzania/TZA_2014_NPS-R4_v03_M_v01_A_EXT_CSV/hh_sec_a.csv")
tz_panel = pd.read_csv("Data/lsms/Tanzania/TZA_2008-2014_NPS-UPD_v01_M_CSV/upd4_hh_a.csv", dtype={'strataid2':'string'})
tz_wv5 = pd.read_csv("Data/lsms/Tanzania/TZA_2019_NPD-SDD_v05_M_CSV/HH_SEC_A.csv")

In [15]:
# get cluster id for each ea in wave 1:
wv1_clusterid = tz_panel[['clusterid','round','r_hhid']].rename(columns = {'r_hhid':'hhid'})
wv1_clusterid = wv1_clusterid.loc[wv1_clusterid['round'] == 1,:].drop(columns = 'round')
wv1_clusterid = wv1_clusterid.loc[~wv1_clusterid.duplicated(keep = 'first'),:]
tz_wv1 = pd.merge(tz_wv1,wv1_clusterid, on = 'hhid', how = 'left')


In [16]:
# enumeration areas
wv1_geos = tz_wv1[['clusterid','lat_modified','lon_modified']]
wv1_geos = wv1_geos.loc[~wv1_geos.duplicated(keep = 'first'),:]
wv1_geos['round'] = 1
print("In wave 1, I have ", len(np.unique(wv1_geos['clusterid'])), 'enumeration areas')
print("In wave 1, I have ", len(wv1_geos), 'different pairs of coordinates')

In wave 1, I have  409 enumeration areas
In wave 1, I have  409 different pairs of coordinates


In [17]:
# enumeration areas wave 2
wv2_geos = tz_wv2[['ea_id','lat_modified','lon_modified']]
wv2_geos = pd.merge(wv2_geos, tz_wv1[['ea_id','clusterid']], on = 'ea_id', how = 'left')
wv2_geos = wv2_geos.loc[~wv2_geos.duplicated(keep = 'first'),:].drop(columns = 'ea_id')
wv2_geos['round'] = 2
print("In wave 2, I have ", len(np.unique(wv2_geos['clusterid'])), 'enumeration areas')
print("In wave 2, I have ", len(wv2_geos), 'different pairs of coordinates')
print("The additional pairs of coordinates derive from the splitoff households")
print("For instance an adult member of the hosuehold moved to Dar es Salam (see map below)")

In wave 2, I have  409 enumeration areas
In wave 2, I have  750 different pairs of coordinates
The additional pairs of coordinates derive from the splitoff households
For instance an adult member of the hosuehold moved to Dar es Salam (see map below)


In [18]:
fig = px.scatter_mapbox(wv2_geos.iloc[0:2,:], lat="lat_modified", lon="lon_modified",size_max=30, zoom=3)
fig.update_layout(mapbox_style="open-street-map")
fig.show()

In [19]:
# keep only the enumeration areas from the first wave
# get the original coordinates for each clusterid
clusterid_lat_dict = dict(zip(wv1_geos['clusterid'],wv1_geos['lat_modified']))
clusterid_lon_dict = dict(zip(wv1_geos['clusterid'],wv1_geos['lon_modified']))

wv2_geos['lat_modified'] = [clusterid_lat_dict[i] for i in list(wv2_geos['clusterid'])]
wv2_geos['lon_modified'] = [clusterid_lon_dict[i] for i in list(wv2_geos['clusterid'])]
wv2_geos = wv2_geos.loc[~wv2_geos.duplicated(keep = 'first'),:]

In [20]:
wv2_geos

,lat_modified,lon_modified,clusterid,round
0,-5.08575,35.854389,1114002,2
72,-6.45662,36.722858,1203003,2
144,-6.61401,36.508659,1217003,2
216,-6.17725,36.475750,1313004,2
280,-6.22962,36.413330,1313301,2
...,...,...,...,...
28144,-5.26196,39.790211,55117023,2
28216,-5.35541,39.650162,55201005,2
28304,-5.40396,39.694229,55209022,2
28360,-5.36837,39.702129,55213303,2


In [21]:
# enumeration areas wave 3
wv3_geos = pd.merge(tz_wv3[['y3_hhid','lat_dd_mod','lon_dd_mod']], tz_wv3_a[['y3_hhid','clusterid']], on = 'y3_hhid', how = 'left')
wv3_geos = wv3_geos.drop(columns = 'y3_hhid')
wv3_geos = wv3_geos.rename(columns = {"lat_dd_mod":'lat_modified', "lon_dd_mod":'lon_modified'})
wv3_geos = wv3_geos.loc[~wv3_geos.duplicated(keep = 'first'),:]
wv3_geos['round'] = 3
print("In wave 3, I have ", len(np.unique(wv3_geos['clusterid'])), 'enumeration areas')
print("In wave 3, I have ", len(wv3_geos), 'different pairs of coordinates')
print("The additional pairs of coordinates derive from the splitoff households")
print("There have been splitoff households in the second wave, these were revisted in the third round.",
     "Thus, these households could also be included in the panel... if I wanted to be very precise.",
     "(but few datapoints in that case so does not really make sense)")

In wave 3, I have  409 enumeration areas
In wave 3, I have  1602 different pairs of coordinates
The additional pairs of coordinates derive from the splitoff households
There have been splitoff households in the second wave, these were revisted in the third round. Thus, these households could also be included in the panel... if I wanted to be very precise. (but few datapoints in that case so does not really make sense)


In [22]:
wv3_geos['lat_modified'] = [clusterid_lat_dict[i] for i in list(wv3_geos['clusterid'])]
wv3_geos['lon_modified'] = [clusterid_lon_dict[i] for i in list(wv3_geos['clusterid'])]
wv3_geos = wv3_geos.loc[~wv3_geos.duplicated(keep = 'first'),:]

In [23]:
# enumeration areas wave 4
wv4_geos = pd.DataFrame({"clusterid": np.unique(tz_wv4['clusterid'])})
wv4_geos = pd.merge(wv4_geos,wv1_geos,on='clusterid',how = 'left')
wv4_geos['round'] = 4
print("In wave 4, for", sum(wv4_geos['lat_modified'].isna()), 'ennumeration areas there is no GPS info')
print("There are in total", len(wv4_geos) - sum(wv4_geos['lat_modified'].isna()),
      'enumeration areas with GPS info in wave 4')
# remove eas without GPS information
wv4_geos = wv4_geos.loc[~wv4_geos['lat_modified'].isna(),:]


In wave 4, for 12 ennumeration areas there is no GPS info
There are in total 56 enumeration areas with GPS info in wave 4


In [24]:
# enumeration areas in wave 5
wv5_clusterid = tz_wv4[['y4_hhid','clusterid']]
# the cluster id in wave 5 again follows some other fucking convention (I fucking hate this!)
wv5_geos = pd.merge(tz_wv5[['y4_hhid']], wv5_clusterid, on = 'y4_hhid', how = 'left')
wv5_geos = wv5_geos.drop(columns = 'y4_hhid')
wv5_geos = wv5_geos.loc[~wv5_geos.duplicated(),:]
wv5_geos = pd.merge(wv5_geos,wv1_geos,on='clusterid',how = 'left')
wv5_geos['round'] = 5
print("In wave 5, for", sum(wv5_geos['lat_modified'].isna()), 'ennumeration areas there is no GPS info')
print("There are in total", len(wv5_geos) - sum(wv5_geos['lat_modified'].isna()),
      'enumeration areas with GPS info in wave 5')
# again remove them...
wv5_geos = wv5_geos.loc[~wv5_geos['lat_modified'].isna(),:]

In wave 5, for 12 ennumeration areas there is no GPS info
There are in total 56 enumeration areas with GPS info in wave 5


In [25]:
# plot the different enumeration areas...
tz_map_data = pd.concat([wv1_geos, wv2_geos, wv3_geos, wv4_geos, wv5_geos], axis = 0)
tz_map_data['round'] = tz_map_data['round'].astype('category')
round_year_dict = {1:2009, 2:2011, 3: 2013, 4: 2015, 5:2019}
tz_map_data['year'] = [round_year_dict[i] for i in tz_map_data['round']]

In [26]:
fig = px.scatter_mapbox(tz_map_data, lat="lat_modified", lon="lon_modified", color = 'round',size_max=30, zoom=3)
fig.update_layout(mapbox_style="open-street-map")
fig.show()

In [27]:
tz_map_data = tz_map_data.reset_index(drop = True)
tz_map_data

,clusterid,lat_modified,lon_modified,round,year
0,1114002,-5.08575,35.854389,1,2009
1,1203003,-6.45662,36.722858,1,2009
2,1217003,-6.61401,36.508659,1,2009
3,1313004,-6.17725,36.475750,1,2009
4,1313301,-6.22962,36.413330,1,2009
...,...,...,...,...,...
1334,20324004,-1.89927,33.401550,5,2019
1335,51107052,-5.84791,39.295750,5,2019
1336,53219008,-6.18554,39.216640,5,2019
1337,54117008,-5.06196,39.726410,5,2019


In [28]:
tz_map_data.groupby('year').size()


year
2009    409
2011    409
2013    409
2015     56
2019     56
dtype: int64

In [29]:
print("56 enumeration areas are available for all 5 waves")
print(409 - 56,"enumeration areas are available for the first three waves.")

56 enumeration areas are available for all 5 waves
353 enumeration areas are available for the first three waves.


In [31]:
tz_geos = tz_map_data

In [32]:
tz_geos

,clusterid,lat_modified,lon_modified,round,year
0,1114002,-5.08575,35.854389,1,2009
1,1203003,-6.45662,36.722858,1,2009
2,1217003,-6.61401,36.508659,1,2009
3,1313004,-6.17725,36.475750,1,2009
4,1313301,-6.22962,36.413330,1,2009
...,...,...,...,...,...
1334,20324004,-1.89927,33.401550,5,2019
1335,51107052,-5.84791,39.295750,5,2019
1336,53219008,-6.18554,39.216640,5,2019
1337,54117008,-5.06196,39.726410,5,2019


# This is older stuff directly using the panel dataset...

Geovariables are only available for waves 1-3. In the panel dataset they are omitted, thus need to reconstruct them from the first wave.
WV4 includes households as well as the panel dataset includes households that were nwely sampled in wv4 (only a subset was sampled in all four periods. However, this subset of households is also included in wv5.
Thus, I remove all households that were only included once in wave 4 (also because I cannot reconstruct the geodata for them).